In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score

data = pd.read_csv("data\matchs_2013_2022.csv", sep=",")
dataAPredire = pd.read_csv("data\match_2023.csv", sep=",")
y = data["results"]
X = data[["home_club_id", "away_club_id"]]
# print(X.head(), y.head())


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model selection
model = RandomForestClassifier()

# Model training
model.fit(X_train, y_train)

# Model evaluation
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))

r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2}')

[[ 77  73  87]
 [ 53  51 100]
 [ 65  89 221]]
              precision    recall  f1-score   support

          -1       0.39      0.32      0.36       237
           0       0.24      0.25      0.24       204
           1       0.54      0.59      0.56       375

    accuracy                           0.43       816
   macro avg       0.39      0.39      0.39       816
weighted avg       0.42      0.43      0.42       816

0.42769607843137253
R² score: -0.5679632266606711
